In [16]:
import numpy as np
import random

In [38]:
class Restaurant:
    def __init__(self, mu, std, name=''):
        self.name = name
        self.mu = mu
        self.std = std
    def sample(self):
        return np.random.normal(self.mu, self.std)

In [30]:
def explore_only(restaurants, days):
    score_total = 0
    for _ in range(days):
        score_total += random.choice(restaurants).sample()
    return score_total

In [31]:
def exploit_only(restaurants, days):
    score_total = 0
    best_rst = None
    best_score = 0
    for rst in restaurants:
        sample = rst.sample()
        if  sample > best_score:
            best_rst = rst
            best_score = sample            
        
    for _ in range(days - len(restaurants)):
        score_total += best_rst.sample()
    
    return score_total

In [83]:
def epsilon_greedy(restaurants, days, epsilon=0.05):
    score_total = 0
    history = {}
    for rst in restaurants:
        history[rst] = [rst.sample()]
    
    for _ in range(days - len(restaurants)):
        rnd = np.random.random()
        # Explore
        if (rnd < epsilon):
            chosen = random.choice(restaurants)
        # Exploit    
        else:
            chosen = sorted(history.items(), key=lambda pair: np.mean(pair[-1]))[-1][0]
        score = chosen.sample()
        score_total += score
        history[chosen].append(score)
    return score_total

In [99]:
def ucb1(restaurants, days):
    score_total = 0
    history = {rst: [rst.sample()] for rst in restaurants}
    
    for t in range(len(restaurants), days):
        chosen = sorted(history.items(), key=lambda pair: np.mean(pair[-1]) + np.sqrt(2 * np.log(t + 1) / len(pair[-1])))[-1][0]
        score = chosen.sample()
        score_total += score
        history[chosen].append(score)
    return score_total

In [39]:
std_factor = 0.5
num_restaurants = 3

mu_vals = [3 * i for i in range(1, num_restaurants + 1)]
std_vals = [mu * std_factor for mu in mu_vals]
mu_std_pairs = zip(mu_vals, std_vals)

candidates = [Restaurant(mu, std) for mu, std in mu_std_pairs]

In [21]:
num_days = 300
optimal_average = max(mu_vals) * num_days
print('Optimal score: {}'.format(optimal_average))

Optimal score: 2700


In [32]:
explore_only_vals = []
for _ in range(1000):
    val = explore_only(candidates, num_days)
    explore_only_vals.append(val)
print('Explore Only Mean Regret: {}'.format((optimal_average - np.mean(explore_only_vals)) / optimal_average))

Explore Only Mean Regret: 0.3340336046298928


In [64]:
exploit_only_vals = []
for _ in range(1000):
    val = exploit_only(candidates, num_days)
    exploit_only_vals.append(val)
print('Exploit Only Mean Regret: %s'%((optimal_average - np.mean(exploit_only_vals)) / optimal_average))

Exploit Only Mean Regret: 0.10676547428258244


In [94]:
epsilon = 0.1
epsilon_greedy_vals = []
for _ in range(1000):
    val = epsilon_greedy(candidates, num_days, epsilon)
    epsilon_greedy_vals.append(val)
print('Epsilon Greedy Mean Regret (epsilon={}): {}'.format(epsilon, (optimal_average - np.mean(epsilon_greedy_vals)) / optimal_average))

Epsilon Greedy Mean Regret (epsilon=0.1): 0.0594137799935064


In [102]:
epsilon = 0.05
epsilon_greedy_vals = []
for _ in range(1000):
    val = epsilon_greedy(candidates, num_days, epsilon)
    epsilon_greedy_vals.append(val)
print('Epsilon Greedy Mean Regret (epsilon={}): {}'.format(epsilon, (optimal_average - np.mean(epsilon_greedy_vals)) / optimal_average))

Epsilon Greedy Mean Regret (epsilon=0.05): 0.057500183342456324


In [101]:
ucb1_vals = []
for _ in range(1000):
    val = ucb1(candidates, num_days)
    ucb1_vals.append(val)
print('UCB1 Mean Regret: {}'.format((optimal_average - np.mean(ucb1_vals)) / optimal_average))

UCB1 Mean Regret: 0.052293375649780365
